# Aggregation and grouping

In [17]:
import numpy as np
import pandas as pd

In [5]:
# generate some random numbers as pd.series
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser


0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [6]:
# sum, mean
ser.sum()

np.float64(2.811925491708157)

In [7]:
ser.mean()

np.float64(0.5623850983416314)

In [8]:
# for dataframe
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [9]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

In [10]:
# aggreate by axis
df.mean(axis=1)

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [11]:
df.describe()

,A,B
count,5.000000,5.000000
mean,0.477888,0.443420
std,0.353125,0.426952
min,0.058084,0.020584
25%,0.155995,0.181825
50%,0.601115,0.212339
75%,0.708073,0.832443
max,0.866176,0.969910


# Group by: split, apply, combine

Split by key (group names), apply a function (sum), combine the results from each group.

In [14]:
df = pd.DataFrame(
    {'key': ['A', 'B', 'C', 'A', 'B', 'C'],
     'data': range(6)},
    columns=['key', 'data']
)
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [15]:
# do group by. this step doesn't print anything
df.groupby('key')

In [18]:
# do aggregate function
df.groupby('key').mean()

,data
key,
A,1.5
B,2.5
C,3.5


## aggregate, filter, transform, apply

In [19]:
# another example
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [22]:
# aggregate method can do multiple computation at once
df.groupby('key').aggregate(['min', 'median', 'max'])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [23]:
# different functions for different columns
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [24]:
# filter
def filter_func(x):
    return x['data2'].std() > 4

In [25]:
# first try to get the std for each column
df.groupby('key').std()

,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


In [27]:
df.groupby('key').filter(filter_func)
# first gets the std for data2, filter on those with bigger than 4, keep the indices, then only return those

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


In [30]:
# transformation
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [33]:
# apply

def plus10(x):
    x['data3'] = x['data1'].mean()
    return x


In [34]:
df.groupby('key').apply(plus10)

/var/folders/b8/rdzjn7jd31b_t6_3txd77stm0000gn/T/ipykernel_29583/1637322208.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('key').apply(plus10)


key  data1  data2  data3
key                           
A   0   A      0      5    1.5
    3   A      3      3    1.5
B   1   B      1      0    2.5
    4   B      4      7    2.5
C   2   C      2      3    3.5
    5   C      5      9    3.5